### Try to find an example to solve the polymatrix game

#### 1. Define the initial setting

In [ ]:
import numpy as np

# 假设有3个玩家，每个玩家有2个可能的动作
num_players = 3
action_space = [2, 2, 2]  # 每个玩家的动作数

# 定义初始的效用矩阵 (这是一个随机矩阵示例)
A_0 = [np.random.rand(action_space[i], action_space[j]) for i in range(num_players) for j in range(num_players) if i != j]

# 初始的效用矩阵，列表形式，A_0[i][j]表示第i个玩家与第j个玩家之间的效用矩阵


#### 2. Define the DSOP

In [101]:
import numpy as np
from scipy.optimize import linprog

def dsop_solution_simple(A_21_0, A_31_0):
    num_vars = 4  # 每个矩阵有2x2 = 4个变量

    # 定义目标函数：最小化绝对值和
    c = np.ones(num_vars * 2)

    # 定义约束条件矩阵和右侧的常数向量
    A_eq = []
    b_eq = []

    # 构建线性化的绝对值约束
    for i in range(2):
        for j in range(2):
            # A(2,1) 矩阵的线性化绝对值约束
            A_eq.append([1 if (p == i and q == j) else 0 for p in range(2) for q in range(2)] + [0, 0, 0, 0])
            b_eq.append(A_21_0[i, j])
            A_eq.append([-1 if (p == i and q == j) else 0 for p in range(2) for q in range(2)] + [0, 0, 0, 0])
            b_eq.append(-A_21_0[i, j])

            # A(3,1) 矩阵的线性化绝对值约束
            A_eq.append([0, 0, 0, 0] + [1 if (p == i and q == j) else 0 for p in range(2) for q in range(2)])
            b_eq.append(A_31_0[i, j])
            A_eq.append([0, 0, 0, 0] + [-1 if (p == i and q == j) else 0 for p in range(2) for q in range(2)])
            b_eq.append(-A_31_0[i, j])

    # 线性规划求解
    res = linprog(c, A_eq=np.array(A_eq), b_eq=np.array(b_eq), bounds=[(0, None)]*num_vars*2, method='highs')

    if res.success:
        A_21_mod = res.x[:4].reshape(2, 2)
        A_31_mod = res.x[4:].reshape(2, 2)

        # 打印初始矩阵
        print("初始矩阵 A^(2,1):")
        print(A_21_0)
        print("初始矩阵 A^(3,1):")
        print(A_31_0)

        # 打印调整后的矩阵
        print("调整后的矩阵 A^(2,1):")
        print(A_21_mod)
        print("调整后的矩阵 A^(3,1):")
        print(A_31_mod)

        # 计算调整的成本（绝对值变化之和）
        cost_A21 = np.sum(np.abs(A_21_mod - A_21_0))
        cost_A31 = np.sum(np.abs(A_31_mod - A_31_0))

        print(f"调整成本 A^(2,1): {cost_A21}")
        print(f"调整成本 A^(3,1): {cost_A31}")
        print(f"总调整成本: {cost_A21 + cost_A31}")

        return A_21_mod, A_31_mod
    else:
        print("线性规划求解失败:", res.message)
        return None

# 示例测试
def test_dsop_solution():
    A_21_0 = np.random.rand(2, 2)
    A_31_0 = np.random.rand(2, 2)

    solution = dsop_solution_simple(A_21_0, A_31_0)
    if solution is not None:
        print("调整完成并打印结果。")
    else:
        print("求解失败。")

test_dsop_solution()


初始矩阵 A^(2,1):
[[0.04077514 0.59089294]
 [0.67756436 0.01658783]]
初始矩阵 A^(3,1):
[[0.51209306 0.22649578]
 [0.64517279 0.17436643]]
调整后的矩阵 A^(2,1):
[[0.04077514 0.59089294]
 [0.67756436 0.01658783]]
调整后的矩阵 A^(3,1):
[[0.51209306 0.22649578]
 [0.64517279 0.17436643]]
调整成本 A^(2,1): 0.0
调整成本 A^(3,1): 0.0
总调整成本: 0.0
调整完成并打印结果。


#### 3. 实现DSECOP策略

线性规划求解失败: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
求解失败。
